In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

# Add the parent directory to the Python path - bad practice, but it's just for the example
import sys
sys.path.append("..")

from src.glocal_analysis import run_analysis 
from src.datasets.flood_dataset import FloodDataset
from src.datasets.DLR_dataset import DatasetDLR
from src.plot_crp_explanations import plot_explanations, plot_one_image_explanation
from src.minio_client import MinIOClient
from LCRP.models import get_model 

/Users/kortukov/miniconda3/envs/tema/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ADDING vgg
ADDING mobilenet_v2
ADDING efficient_net-b3
ADDING mobilenet_v3
ADDING SSDBoxPredictor
ADDING SSDLiteBoxPredictor
ADDING SSDBoxHead


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
device

'cpu'

In [7]:
model = get_model(model_name="unet", device=device, classes=1, ckpt_path="../models/model_s2burnt_modified.pt", in_channels=4, encoder_name="resnet50")

Loaded checkpoint ../models/model_s2burnt_modified.pt


In [7]:
!ls ../../DLR_data/dlr_germany/

S2A_MSIL2A_20210718T103031_R108_T31UGS_20210719T012705
S2A_MSIL2A_20210721T104031_R008_T31UGS_20210721T214914


In [17]:
dlr_germany_image1 = "../../DLR_data/dlr_germany/S2A_MSIL2A_20210718T103031_R108_T31UGS_20210719T012705" 
dlr_germany_image2 = "../../DLR_data/dlr_germany/S2A_MSIL2A_20210721T104031_R008_T31UGS_20210721T214914"

In [18]:
img_dir = dlr_germany_image1

In [19]:
dataset = DatasetDLR(
    img_dir=img_dir,
    mask_dir=None,
    normalize_means_stds=[
        [0.1161, 0.1065, 0.1036, 0.2059],  # Means
        [0.0556, 0.0570, 0.0772, 0.1033]   # Stds
    ]
)

Loading images: 100%|██████████| 1/1 [00:00<00:00, 15.88it/s]


In [20]:
dataset.img_arr.shape

(48, 256, 256, 4)